In [1]:
from sqlalchemy.orm import DeclarativeBase

In [2]:
class Base(DeclarativeBase): pass

In [3]:
from typing import Optional
from datetime import datetime
from sqlalchemy import String, Integer, Identity, DateTime, func
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column

In [4]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [5]:
class IdentityMixin:
    id: Mapped[int] = mapped_column(
        Integer,
        Identity(start=1, increment=1),
        primary_key=True
    )

In [6]:
class TimestampMixin:
    created_at: Mapped[datetime] = mapped_column(
        DateTime(timezone=True),
        server_default=func.now(),
        nullable=False
    )
    lastupdate_at: Mapped[Optional[datetime]] = mapped_column(
        DateTime(timezone=True),
        server_default=func.now(),
        onupdate=func.now()
    )

In [7]:
class Product(IdentityMixin, TimestampMixin, Base):
    __tablename__ = "products"

    name: Mapped[str] = mapped_column(
        String(100),
        nullable = False
    )
    sku: Mapped[str] = mapped_column(
        String(100),
        nullable = True
    )

    # Relationships
    stocks: Mapped[list["Stock"]] = relationship(
        "Product", back_populates="product", cascade="all, delete-orphan"
    )

In [8]:
class Supplier(IdentityMixin, TimestampMixin, Base):
    __tablename__ = "suppliers"

    name : Mapped[str] = mapped_column(
        String(100),
        nullable = False
    )

    contact_name: Mapped[str] = mapped_column(
        String(100),
        nullable = False
    )

    phone : Mapped[str] = mapped_column(
        String(10),
        nullable = False
    )

    email : Mapped[str] = mapped_column(
        String(100),
        nullable = False
    )

    address : Mapped[str] = mapped_column(
        String(256),
        nullable = False
    )



In [9]:
class Warehouse(IdentityMixin, TimestampMixin, Base):
    __tablename__ = "warehouses"
    
    name : Mapped[str] = mapped_column(
        String(100),
        nullable = False
    )

    location: Mapped[str] = mapped_column(
        String(100),
        nullable = False
    )

    # Relationships
    stocks: Mapped[list["Stock"]] = relationship(
        "Warehouse", back_populates="warehouse", cascade="all, delete-orphan"
    )

In [10]:
class Stock(IdentityMixin, TimestampMixin, Base):
    __tablename__ = "stocks"

    product_id: Mapped[int] = mapped_column(ForeignKey("products.id"), nullable=False)
    warehouse_id: Mapped[int] = mapped_column(ForeignKey("warehouses.id"), nullable=False)
    quantity: Mapped[int] = mapped_column(
        Integer,
        default=0
    )

    # Relationship
    warehouse: Mapped["Warehouse"] = relationship("Warehouse", back_populates="stocks")
    product: Mapped["Product"] = relationship("Product", back_populates="stocks")

In [11]:
import os
CONNECTION_URL = os.getenv('INVDB_URL', 'sqlite:///inventory.db')

In [12]:
from sqlalchemy import create_engine
engine = create_engine(url = CONNECTION_URL, echo=True)

In [13]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import  relationship

class User(IdentityMixin, TimestampMixin, Base):
    __tablename__ = "users"
    username: Mapped[str] = mapped_column(
        String(100),
        nullable = False
    )

    # logical field
    posts: Mapped[list["Post"]] = relationship(
        "Post", back_populates="user", cascade="all, delete-orphan"
    )

In [14]:


class Post(IdentityMixin, TimestampMixin, Base):
    __tablename__ = "posts"
    post:  Mapped[str] = mapped_column(
        String(100),
        nullable = False
    )
    user_id: Mapped[int] = mapped_column(
        Integer,
        ForeignKey('users.id'),
        nullable=False
    )
    # logical field
    user: Mapped["User"] = relationship("User", back_populates="posts")

In [15]:
# Student model
class Student(IdentityMixin, TimestampMixin, Base):
    __tablename__ = "students"

    name: Mapped[str] = mapped_column(String(50), nullable=False)
    enrollments: Mapped[list["Enrollment"]] = relationship(
        "Enrollment", back_populates="student", cascade="all, delete-orphan"
    )


class Course(IdentityMixin, TimestampMixin, Base):
    __tablename__ = "courses"

    title: Mapped[str] = mapped_column(String(100), nullable=False)
    enrollments: Mapped[list["Enrollment"]] = relationship(
        "Enrollment", back_populates="course", cascade="all, delete-orphan"
    )

In [16]:
class Enrollment(IdentityMixin, TimestampMixin, Base):
    __tablename__ = "enrollments"

    student_id: Mapped[int] = mapped_column(ForeignKey("students.id"), nullable=False)
    course_id: Mapped[int] = mapped_column(ForeignKey("courses.id"), nullable=False)

    
    # Relationships
    student: Mapped["Student"] = relationship("Student", back_populates="enrollments")
    course: Mapped["Course"] = relationship("Course", back_populates="enrollments")

In [17]:
Base.metadata.create_all(engine)

2025-10-19 10:48:21,380 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-19 10:48:21,382 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2025-10-19 10:48:21,383 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-19 10:48:21,386 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("products")
2025-10-19 10:48:21,388 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-19 10:48:21,390 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("suppliers")
2025-10-19 10:48:21,391 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-19 10:48:21,392 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("suppliers")
2025-10-19 10:48:21,392 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-19 10:48:21,393 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("warehouses")
2025-10-19 10:48:21,394 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-19 10:48:21,394 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("warehouses")
2025-10-19 10:48:21,395 INFO sqlalchemy.engine.Engine 